# Computing thermodynamic properties

## Standard thermodynamic properties of species

For species, we can compute the following *standard thermodynamic properties*:

* the standard molar Gibbs energy, $G_i^\circ$
* the standard molar Helmholtz energy, $A_i^\circ$
* the standard molar enthalpy, $H_i^\circ$
* the standard molar internal energy, $U_i^\circ$
* the standard molar entropy, $S_i^\circ$
* the standard molar volume, $V_i^\circ$
* the standard molar heat capacity (constant pressure), $C_{P,i}^\circ$
* the standard molar heat capacity (constant volume), $C_{V,i}^\circ$

Below, we use the SUPCRTBL database to compute the standard thermodynamic properties of CO<sub></sub>(aq).

In [ ]:
from reaktoro import *

db = SupcrtDatabase("supcrtbl")

CO2aq   = db.species("CO2(aq)")

We can use the method `props` in the [Species](https://reaktoro.org/api/classReaktoro_1_1Species.html) class to compute the standard thermodynamic properties of instance `CO2aq` at 60 °C and 100 bar:

In [ ]:
print("STANDARD THERMODYNAMIC PROPERTIES OF CO2(AQ) AT 60 °C AND 100 BAR")
print(CO2aq.props(60, "C", 100, "bar"))

**TASK 1**: compute the standard thermodynamic properties of CO(g) and calcite.

In [ ]:
# -------------------------------------- #
# Code cell for the task
# -------------------------------------- #

Let's calculate the *standard thermodynamic properties of CO<sub></sub>(aq) from 25 to 300 °C* along the *saturation pressure of water*. Below, we build a dictionary containing the following data:
* the temperatures used to evaluate thermodynamic properties
* the standard molar Gibbs energy of CO<sub></sub>(aq)
* the standard molar enthalpy of CO<sub></sub>(aq)

In [ ]:
import numpy as np

temperatures = np.linspace(25.0, 300.0, 100) + 273.15  # in K

data = { "T": [], "G0": [], "H0": [] }

for T in temperatures:
    P = waterSaturationPressureWagnerPruss(T)   # in Pa
    props = CO2aq.props(T, P)                   # properties calculated in K and Pa
    data["T" ].append(float(T - 273.15))        # in °C
    data["G0"].append(float(props.G0 * 0.001))  # in kJ/mol
    data["H0"].append(float(props.H0 * 0.001))  # in kJ/mol

> **Tip:** You might also be interested in other methods for calculating the thermodynamic properties of water besides `waterSaturationPressureWagnerPruss` (which implements the water saturation pressure equation). Below, we list other methods available in [Reaktoro's API](https://reaktoro.org/api/):
>
> * `waterDensityHGK`
> * `waterDensityWagnerPruss`
> * `waterLiquidDensityHGK`
> * `waterLiquidDensityWagnerPruss`
> * `waterVaporDensityHGK`
> * `waterVaporDensityWagnerPruss`
> * `waterPressureHGK`
> * `waterPressureWagnerPruss`
> * `waterSaturationPressureWagnerPruss`
> * `waterSaturationLiquidDensityWagnerPruss`
> * `waterSaturationVapourDensityWagnerPruss`
> * `waterThermoPropsHGK`
> * `waterThermoPropsWagnerPruss`

We use the [bokeh](https://bokeh.org/) plotting library, which must be initialized to work in Jupyter Notebooks:

In [ ]:
from bokeh.plotting import figure, show
from bokeh.models import HoverTool
from bokeh.io import output_notebook
output_notebook()

The interactive plot of standard molar Gibbs energy and standard molar enthalpy versus temperature range is build below:

In [ ]:
hovertool = HoverTool()
hovertool.tooltips = [("Temperature", "@T °C"), ("G°,CO2(aq)", "@G0 kJ/mol"), ("H°,CO2(aq)", "@H0 kJ/mol")]

p = figure(
    title="STANDARD THERMODYNAMIC PROPERTIES OF CO2(AQ)\nALONG WATER SATURATION PRESSURE", 
    x_axis_label='TEMPERATURE [°C]', 
    y_axis_label=r"",
    sizing_mode="scale_width")

p.add_tools(hovertool)

p.line("T", "G0", source=data, legend_label="G°,CO2(aq)", line_width=5, line_cap="round", line_color="midnightblue")
p.line("T", "H0", source=data, legend_label="H°,CO2(aq)", line_width=5, line_cap="round", line_color="orange")

show(p)

**TASK 2**: use the PHREEQC database to collect the *standard molar internal energy* and *standard molar entropy of CO2* and plot their temperature dependence.

In [ ]:
# -------------------------------------- #
# Code cell for the task
# -------------------------------------- #

## Standard thermodynamic properties of reactions

Next, we demonstrate computation of *standard thermodynamic properties of a chemical reaction* such as:

* the equilibrium constant of reaction, $K_r$
* the standard Gibbs energy of reaction, $\Delta G_r^\circ$
* the standard Helmholtz energy of reaction, $\Delta A_r^\circ$
* the standard enthalpy of reaction, $\Delta H_r^\circ$
* the standard internal energy of reaction, $\Delta U_r^\circ$
* the standard entropy of reaction, $\Delta S_r^\circ$
* the standard volume of reaction, $\Delta V_r^\circ$
* the standard heat capacity (constant pressure) of reaction, $\Delta C_{P,r}^\circ$
* the standard heat capacity (constant volume) of reaction, $\Delta C_{V,r}^\circ$

Reaktoro computes these reaction properties using the standard thermodynamic properties of the reacting species at a given temperature and pressure.

**Example**: for the reaction $2\mathrm{A}+3\mathrm{B}=\mathrm{C}$, the *standard Gibbs energy of this reaction* is computed using

$$\Delta G_{r}^{\circ}=G_{\mathrm{C}}^{\circ}-2G_{\mathrm{A}}^{\circ}-3G_{\mathrm{B}}^{\circ}.$$

This rationale (stoichiometric property contribution from product species minus the contribution of reactant species) can be applied to all other standard thermodynamic properties and every reaction.

The *equilibrium constant of the reaction* is defined as

$$\ln K_r = -\dfrac{\Delta G^{\circ}_m}{RT}$$

but it's often handled in logarithm base 10, $\lg K_r = \ln K_r/\ln 10$.

Below, we demonstrate computing of these reaction properties:

In [ ]:
from reaktoro import *

db = PhreeqcDatabase("phreeqc.dat")

rxn = db.reaction("H+ + OH- = H2O")

rprops = rxn.props(25.0, "C", 1.0, "atm")
print(rprops)

**TASK 3**: consider reaction of kaolinite dissolution `Kaolinite + 6*H+ = H2O + 2*H4SiO4 + 2*Al+3` and compute its properties.

In [ ]:
# -------------------------------------- #
# Code cell for the task
# -------------------------------------- #